In [23]:
import json
import numpy as np

# Load the JSON data
with open("data.json", "r") as file:
    data = json.load(file)

# Extracting data from the JSON structure
t = np.array(
    data["t"]
)  # Time steps, might not be used directly in LSTM but useful for splitting data
o = np.array(data["o"])  # Open prices
h = np.array(data["h"])  # High prices
l = np.array(data["l"])  # Low prices
c = np.array(data["c"])  # Close prices
v = np.array(data["v"])  # Volumes

In [24]:
# Assuming you want to use 'o', 'h', 'l', 'c', 'v' from day 't-1' to predict 'o', 'h', 'l', 'c', 'v' for day 't'
# Here's how you can prepare your data:

# Features for model input
features = np.column_stack((o[:-1], h[:-1], l[:-1], c[:-1], v[:-1]))

# Targets for model output
targets = np.column_stack((o[1:], h[1:], l[1:], c[1:], v[1:]))

# Reshape for LSTM [samples, time steps, features]
# In this case, each day is one time step with 5 features
features = features.reshape((features.shape[0], 1, features.shape[1]))

In [25]:
from sklearn.model_selection import train_test_split

# Splitting data into training and testing (assuming you're using 80-20 split)
train_features, test_features, train_targets, test_targets = train_test_split(
    features, targets, test_size=0.2, random_state=42
)

In [35]:
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(
    LSTM(
        units=50,
        activation="relu",
        return_sequences=True,
    )
)
model.add(Dropout(0.2))


model.add(LSTM(units=60, activation="relu", return_sequences=True))
model.add(Dropout(0.3))


model.add(LSTM(units=80, activation="relu", return_sequences=True))
model.add(Dropout(0.4))


model.add(LSTM(units=120, activation="relu"))
model.add(Dropout(0.5))

model.add(Dense(units=1))

model.compile(optimizer="adam", loss="mean_squared_error", metrics=["MAE"])

In [36]:
model.fit(train_features, train_targets, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
17/17 [==============================] - 13s 412ms/step - loss: 860742221824.0000 - MAE: 289628.3750 - val_loss: 791147053056.0000 - val_MAE: 263958.8438
Epoch 2/10
17/17 [==============================] - 1s 79ms/step - loss: 858914881536.0000 - MAE: 288688.0000 - val_loss: 788758462464.0000 - val_MAE: 262130.5312
Epoch 3/10
17/17 [==============================] - 2s 91ms/step - loss: 855793467392.0000 - MAE: 288061.6562 - val_loss: 784660561920.0000 - val_MAE: 261933.6875
Epoch 4/10
17/17 [==============================] - 1s 81ms/step - loss: 844082839552.0000 - MAE: 292055.6875 - val_loss: 770420965376.0000 - val_MAE: 267152.7188
Epoch 5/10
17/17 [==============================] - 3s 150ms/step - loss: 814116569088.0000 - MAE: 310846.2188 - val_loss: 740110630912.0000 - val_MAE: 286755.8438
Epoch 6/10
17/17 [==============================] - 1s 79ms/step - loss: 776692498432.0000 - MAE: 350816.1562 - val_loss: 702974525440.0000 - val_MAE: 320546.7812
Epoch 7/10
17/17 [=

In [37]:
last_day_data = np.array([[o[-1], h[-1], l[-1], c[-1], v[-1]]])
last_day_features = last_day_data.reshape((1, 1, 5))

# Assuming 'last_day_features' is your input data for the last available day shaped correctly
# For example: last_day_features = np.array([[[o, h, l, c, v]]])

predictions = model.predict(last_day_features)

# 'predictions' will contain the predicted 'o', 'h', 'l', 'c', 'v' values for the next day
print(predictions)

1/1 [==============================] - 2s 2s/step
[[177176.34]]
